Merging of an Excel class spreadsheet downloaded from the HUB with a text file of e-mail addresses 

In [64]:
import openpyxl

students_file = "grades_MTH309Y.xlsx"
sheet = "students"
emails_col = "F" #new column where e-mail addresses will be entered
emails_col_num = openpyxl.utils.column_index_from_string(emails_col)

emails_file = "emails309.txt" # text file with e-mail addresses separated by commas

with open(emails_file, 'r') as f:
    emails = f.read().replace(' ', '').split(',')

wbook = openpyxl.load_workbook(students_file, data_only=True)
wsheet = wbook[sheet]

wsheet[emails_col + "1"].value = "Emails"

for r, e in zip(wbook[sheet].iter_rows(row_offset = 1), emails):
    r[emails_col_num -1].value = e

wbook.save(filename = "test.xlsx")

/Users/bb1/anaconda3/lib/python3.6/site-packages/openpyxl/reader/worksheet.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Sent e-mails

In [ ]:
from email.message import EmailMessage
import smtplib

def send_emails(instructor_password, num):
    
    student_file = "../grades{}.xlsx"
    sheet = "students"
    
    grades = openpyxl.load_workbook(student_file, data_only=True, read_only = True)[sheet] 
    

    ubit_col = 1
    name_col = 2
    row_offset = 2
    grade_cols = (3, 10)
    
    instructor_email = "badzioch@buffalo.edu"
    
    instructor_name = "BB"
    
    msg_txt = ("Dear {fname},\n\n"
               "I am attaching an html file containing your graded MTH 309 homework assignment {num}. "
               "You can open it in any web browser.\n\n"
               "Best regards,\n\n"
               "{instructor_name}\n\n")

    
    server = smtplib.SMTP_SSL('smtp.buffalo.edu', 465)
    server.login(instructor_email, instructor_password)
    
    
    for r in grades.iter_rows(row_offset=row_offset):
        if r[ubit_col].value == None:
            continue
        else:
            name = r[name_col].value
            ubit = r[ubit_col].value
            
            s_email = ubit + "@buffalo.edu"
            text = msg_txt.format(fname = name, num = num, instructor_name = instructor_name)
            html_file = "{}_hw_{}_graded.html".format(ubit, num)
            html_path = graded_path + html_file
            if not os.path.isfile(html_path):
                print('>{:30} *********** FILE NOT FOUND'.format(name))
                continue 
        
            msg = EmailMessage()
            msg['Subject'] = "MTH 309 homework {}".format(num)
            msg['From'] = instructor_email
            msg['To'] = s_email
            
            msg['Content-Type'] = "text/plain; charset=utf-8; format=flowed"
            msg.set_content(text)
                        
            with open(html_path, 'rb') as f:
                msg.add_attachment(f.read(), maintype = "text", subtype = "html", filename = html_file)
                
            body = msg.get_body()
            body.replace_header('Content-Type', 'text/plain; charset=utf-8; format=flowed')
    
            for p in msg.walk():
                if p['Content-Type'] == 'text/html':
                    p.replace_header('Content-Type', 'text/html; name={}'.format(html_file))
                    break
            try:
                #change to_addrs
                server.send_message(msg, from_addr = instructor_email, to_addrs = instructor_email)
            except (smtplib.SMTPRecipientsRefused, 
                    smtplib.SMTPHeloError, 
                    smtplib.SMTPSenderRefused, 
                    smtplib.SMTPDataError, 
                    smtplib.SMTPNotSupportedError):
                print('{:30} *********** NOT SENT'.format(name))
            
            else:
                print('{:30} *********** SENT'.format(name))
    
    server.quit()